Task: Data extraction through API (data source: Electrica De Espana)
Source: https://www.ree.es/es/apidatos

In [ ]:
import pandas as pd
import requests
import json
import pymysql as py

In [ ]:
def mix_generation(start_date, end_date):
    #Enter the start_date and end_date in the following format: YYYY-MM-DDTHH:MM
    
    # API get request
    response = requests.get(f"https://apidatos.ree.es/en/datos/generacion/estructura-generacion?start_date={start_date}&end_date={end_date}&time_trunc=day")
    response_json = response.json()
    data = response_json

    # We drop the info that we do not need
    data = data["included"]
    
    # Create an empty DataFrame with the info that we need
    df = pd.DataFrame(columns=['Type', 'Value', 'Date', 'Percentage'])
    
    # For each type we are accessing to convert the values into dataframe that will be concatenated with our empty df
    for t in data:
        desc = t["type"]
        values_list = t["attributes"]["values"]
        df_values = (pd.DataFrame(values_list)
                     .assign(Type=desc)
                     .rename(columns={"type": "Type",
                                      "value": "Value",
                                      "datetime": "Date",
                                      "percentage": "Percentage"}))
        df = pd.concat([df, df_values])

    # Drop the columns that we do not need
    df = df.reset_index(drop=True)

    # Modify the date type
    df['Date'] = pd.to_datetime(df["Date"],format='%Y-%m')
    return df

In [ ]:
# Get the data for 2020 and 2019
mix_20 = mix_generation("2020-01-01T00:00", "2020-06-15T00:00")
mix_19 = mix_generation("2019-01-01T00:00", "2019-06-15T00:00")

In [ ]:
# Export the data in a .csv format
mix_20 = mix_20.to_csv("clean_db/Mix-Generation_2020.csv", index=False)
mix_19 = mix_19.to_csv("clean_db/Mix-Generation_2019.csv", index=False)